In [1]:
import velvet as vt

# general packages
import numpy as np
import pandas as pd
import torch
from scipy.sparse import issparse

# velocity packages
import scanpy as sc
import scvelo as scv
import anndata as ann

# plotting packages
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, trange
from IPython.display import clear_output

# color palette object
from colors import colorpalette as colpal

Global seed set to 0
/camp/home/maizelr/.local/lib/python3.10/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/camp/home/maizelr/.local/lib/python3.10/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from eval_functions import unitvelo_cross_boundary_correctness as cross_boundary_correctness
from  eval_functions import unitvelo_inner_cluster_coh as inner_cluster_coh

In [26]:
dataset = ['mini_V3', 'mini_MN', 'mini_MD',
           'midi_NM', 'midi_Ne', 'maxi']

scores = np.zeros(len(dataset))
for i, name in enumerate(dataset):
    adata = sc.read_h5ad(f'../../data/benchmarking/{name}.h5ad')
    cr = adata.obsm['velocity_cr_pca'][:,:50]
    pt = adata.obsm['velocity_pst'][:,:50]
    score = np.diagonal(cosine_similarity(cr, pt)).mean()
    scores[i] = score

np.save('../../output_data/benchmarking_scores/positive_control.npy', scores)

In [55]:
def CR_pipeline(adata, name):
    CR = adata.obsm['velocity_cr_pca'][:,:50]
    PCs = adata.varm['precomputed_PCs']
    X_pca = adata.obsm['X_pca'][:,:50]
    Y_pca = adata.obsm['X_pca'][:,:50] + CR
    
    X_hat = X_pca @ PCs[:,:50].T
    Y_hat = Y_pca @ PCs[:,:50].T
    V_hat = Y_hat - X_hat
    
    return V_hat

def PT_pipeline(adata, name):
    PT = adata.obsm['velocity_pst'][:,:50]
    PCs = adata.varm['precomputed_PCs']
    X_pca = adata.obsm['X_pca'][:,:50]
    Y_pca = adata.obsm['X_pca'][:,:50] + PT
    
    X_hat = X_pca @ PCs[:,:50].T
    Y_hat = Y_pca @ PCs[:,:50].T
    V_hat = Y_hat - X_hat
    
    return V_hat

In [53]:
def gene_specific_benchmark(output_folder, pipeline, pipeline_name):
    gene_results = {
        'mini_MN':[
            ['leiden','4','Olig2','-'],
            ['leiden','4','Tubb3','+'],
            ['leiden','2','Neurog2','+'],
            ['leiden','3','Isl2','+'],
        ],
        'mini_V3':[
            ['leiden','1','Sim1','+'],
            ['leiden','1','Sox2','-'],
            ['leiden','3','Tubb3','+'],
            ['leiden','1','Map2','+'],
        ],
        'mini_MD':[
            ['leiden','3','Sox2','-'],
            ['leiden','3','Nkx1-2','-'],
            ['leiden','3','T','-'],
            ['leiden','2','Meox1','+'],
        ],
        'midi_NM':[
            ['cell_annotation','Neural','Olig2','+'],
            ['cell_annotation','Neural','T','-'],
            ['cell_annotation','Mesoderm','Meox1','+'],
            ['cell_annotation','Early_Neural','Irx3','+'],
        ],
        'midi_Ne':[
            ['cell_annotation','Neural','Olig2','+'],
            ['cell_annotation','FP','Shh','+'],
            ['cell_annotation','P3','Nkx2-2','+'],
            ['cell_annotation','pMN','Irx3','-'],
        ]
    }
    
    scores = []
    for name, settings in gene_results.items():
        print(f"GENE SCORE: {name}")
        adata = sc.read_h5ad(f'../../data/benchmarking/{name}.h5ad')
        adata.layers['velocity'] = pipeline(adata, name)

        for seti in settings:
            sub = adata[adata.obs[seti[0]]==seti[1]]
            vel = sub[:,seti[2]].layers['velocity'].flatten()
            if seti[3]=='-':
                score = np.mean(vel<0)
            elif seti[3]=='+':
                score = np.mean(vel>0)
            scores.append(score)
           
    scores = np.array(scores)
    np.save(f'{output_folder}/{pipeline_name}_gene_specific_scores.npy', scores)

In [56]:
print("CR...")
gene_specific_benchmark(
    output_folder='../../output_data/benchmarking_scores', 
    pipeline=CR_pipeline, 
    pipeline_name='CR_POSCONTROL'
)

print("PT...")
gene_specific_benchmark(
    output_folder='../../output_data/benchmarking_scores', 
    pipeline=PT_pipeline, 
    pipeline_name='PT_POSCONTROL'
)

CR...
GENE SCORE: mini_MN
GENE SCORE: mini_V3
GENE SCORE: mini_MD
GENE SCORE: midi_NM
GENE SCORE: midi_Ne
PT...
GENE SCORE: mini_MN
GENE SCORE: mini_V3
GENE SCORE: mini_MD
GENE SCORE: midi_NM
GENE SCORE: midi_Ne


In [11]:
def get_details(name):
    if name == "mini_MN":
        obs = 'leiden'
        cluster_edges = [
            ('0','2'),
            ('2','4'),
            ('2','3'),
            ('4','5'),
        ]
    elif name == "mini_V3":
        obs = 'leiden'
        cluster_edges = [
            ('0','1'),
            ('1','3'),
            ('3','2'),
            ('2','4')
        ]
    elif name == "mini_MD":
        obs = 'leiden'
        cluster_edges = [
            ('2','3'),
            ('3','0'),
            ('3','5'),
            ('0','1'),
            ('1','4'),
        ]
    elif name == "midi_NM":
        obs = 'cell_annotation'
        cluster_edges = [
            ('Early_Neural','Neural'),
            ('NMP','Early_Neural'),
            ('NMP','Mesoderm')
        ]
    elif name == 'midi_Ne':
        obs = 'cell_annotation'
        cluster_edges = [
            ('Neural','pMN'),
            ('pMN','MN'),
            ('pMN','p3'),
            ('p3','V3')
        ]
    elif name == 'maxi':
        obs = 'cell_annotation'
        cluster_edges = [
            ('Early_Neural','Neural'),
            ('NMP','Early_Neural'),
            ('NMP','Mesoderm'),
            ('Neural','pMN'),
            ('pMN','MN'),
            ('pMN','p3'),
            ('p3','V3')
        ]
        
    return obs, cluster_edges


In [20]:
dataset = ['mini_V3', 'mini_MN', 'mini_MD',
           'midi_NM', 'midi_Ne', 'maxi']

results = []
for i, name in enumerate(tqdm(dataset)):
    adata = sc.read_h5ad(f'../../data/benchmarking/{name}.h5ad')
    
    for vel in ['velocity_cr_pca','velocity_pst']:
        x_pca = adata.obsm['X_pca']
        test = ann.AnnData(X=adata.X, obs=adata.obs, var=adata.var,
                           layers={'total':adata.layers['total']})
        test.obsm['X_pca'] = x_pca[:,:50]
        test.obsm['velocity_pca'] = adata.obsm[vel][:,:50]
        scv.pp.neighbors(test) 

        obs, cluster_edges = get_details(name)

        cbd = cross_boundary_correctness(
            test,
            k_cluster=obs,
            k_velocity='velocity',
            x_emb='X_pca',
            cluster_edges=cluster_edges
        )[1]
        results.append(cbd)
    


  0%|          | 0/6 [00:00<?, ?it/s]

computing neighbors
    finished (0:00:02) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing neighbors
    finished (0:00:01) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)


 17%|█▋        | 1/6 [00:07<00:39,  7.81s/it]

computing neighbors
    finished (0:00:01) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing neighbors
    finished (0:00:01) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)


 33%|███▎      | 2/6 [00:13<00:26,  6.71s/it]

...

 67%|██████▋   | 4/6 [00:45<00:27, 13.67s/it]

computing neighbors
    finished (0:00:05) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing neighbors
    finished (0:00:05) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)


 83%|████████▎ | 5/6 [01:08<00:16, 16.72s/it]

computing neighbors
    finished (0:00:11) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing neighbors
    finished (0:00:11) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)


100%|██████████| 6/6 [01:55<00:00, 19.33s/it]


In [24]:
np.save('../../output_data/benchmarking_scores/CBD_PC.npy', np.array(results))